In [ ]:
import os
import numpy as np
import utilities
import argparse
import pickle


from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import *
from sklearn import metrics as skmetrics
from sklearn.metrics import average_precision_score
from sklearn.svm import SVC as SVM
from keras.models import *
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.optimizers import *
from keras.regularizers import *
from keras import metrics
from keras import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
path_save = "/local_mimic/save"
path_tables = "/local_mimic/tables"
path_views = "/local_mimic/views"
path_chkpt = "/local_mimic/save/checkpoint"

Linear Regression Preparation

In [ ]:
onehot = np.load(path_save + '/onehot.npy')
with open(path_save + '/y', 'rb') as f:
    labels = pickle.load(f)

In [ ]:
dct = {'mort':0, 'readmit': 1, 'los': 2, 'dx':3 }
y = np.array([yy[dct['mort']] for yy in labels])

In [ ]:
onehot.shape

In [ ]:
X = onehot

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
penalty = 'l2'
C=0.01533
logreg = LR(penalty=penalty, C=C, random_state=42)
logreg.fit(X_train, y_train)

In [ ]:
filename = path_chkpt + '/Mortality/lr_best_model'
pickle.dump(logreg, open(filename, 'wb'))

In [ ]:
prediction_probabilities = logreg.predict_proba(X_test)[::,1]

In [ ]:
label_prediction =logreg.predict(X_test)

In [ ]:
cnf_matrix = skmetrics.confusion_matrix(y_test, label_prediction)
cnf_matrix

In [ ]:
print("Accuracy:",skmetrics.accuracy_score(y_test, label_prediction))
print("Precision:",skmetrics.precision_score(y_test, label_prediction))
print("Recall:",skmetrics.recall_score(y_test, label_prediction))

In [ ]:
fpr, tpr, _ = skmetrics.roc_curve(y_test,  prediction_probabilities)
auc = skmetrics.roc_auc_score(y_test, prediction_probabilities)

In [ ]:
print("AUC:",auc)

In [ ]:
average_precision_score(y_test, prediction_probabilities)

SVM Preparation

In [ ]:
svm_model = SVM(kernel = 'linear', probability=True)

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
filename = path_chkpt + '/Mortality/svm_best_model'
pickle.dump(svm_model, open(filename, 'wb'))

In [ ]:
prediction_probabilities = svm_model.predict_proba(X_test)[::,1]

In [ ]:
label_prediction =svm_model.predict(X_test)

In [ ]:
cnf_matrix = skmetrics.confusion_matrix(y_test, label_prediction)
cnf_matrix

In [ ]:
print("Accuracy:",skmetrics.accuracy_score(y_test, label_prediction))
print("Precision:",skmetrics.precision_score(y_test, label_prediction))
print("Recall:",skmetrics.recall_score(y_test, label_prediction))

In [ ]:
fpr, tpr, _ = skmetrics.roc_curve(y_test,  prediction_probabilities)
auc = skmetrics.roc_auc_score(y_test, prediction_probabilities)

In [ ]:
print("AUC:",auc)

In [ ]:
average_precision_score(y_test, prediction_probabilities)

LSTM Preparation

In [ ]:
X = np.array(np.load(path_save + '/x19.npy'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
input_shape = (X.shape[1], X.shape[-1])

In [ ]:
filename = path_chkpt + '/Mortality/lstm_best_model'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [ ]:
def lstm_model(input_shape, hidden = 256, targets = 1, learn_rate = 1e-4):
    model = Sequential()
    model.add(Bidirectional(LSTM(hidden), merge_mode = 'concat'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(targets))
    model.add(Activation ('sigmoid'))
    model.compile(loss='binary_crossentropy', 
    optimizer=Adam(lr=learn_rate, beta_1 =.5 ), metrics=['accuracy'])
    return (model)

In [ ]:
filename = path_chkpt + '/Mortality/lstm_best_model'
lstm_model = tf.keras.models.load_model(filename)

In [ ]:
lstm_model = lstm_model(input_shape=input_shape, targets=1, hidden = 256, learn_rate=1e-4)

In [ ]:
lstm_model.fit(x = X, y= y, validation_split=.2, epochs = 100, verbose=2, batch_size=32, callbacks=[checkpoint])

In [ ]:
prediction = lstm_model.predict_proba(X_test,batch_size=32)

In [ ]:
fpr, tpr, _ = skmetrics.roc_curve(y_test,  prediction)
auc = skmetrics.roc_auc_score(y_test, prediction)

In [ ]:
print("AUC:",auc)

In [ ]:
average_precision_score(y_test, prediction)

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

TCN Preparation

In [ ]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
import pickle
from sklearn.model_selection import *
from sklearn import metrics as skmetrics
from sklearn.metrics import average_precision_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tcn import TCN

In [ ]:
path_save = "/local_mimic/save"
path_tables = "/local_mimic/tables"
path_views = "/local_mimic/views"
path_chkpt = "/local_mimic/save/checkpoint"

In [ ]:
X = np.array(np.load(path_save + '/x19.npy'))
with open(path_save + '/y', 'rb') as f:
    labels = pickle.load(f)

In [ ]:
dct = {'mort':0, 'readmit': 1, 'los': 2, 'dx':3 }
y = np.array([yy[dct['mort']] for yy in labels])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
batch_size = 32
maxlen = 24
max_features = 19714

print('Loading data...')
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny))

print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_shape=(456,)))
model.add(TCN(
    nb_filters=64,
    kernel_size=6,
    dilations=[1, 2, 4, 8, 16, 32, 64]
))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

metrics = ['accuracy',
          tf.keras.metrics.AUC(),
          tf.keras.metrics.Precision()]

model.compile('adam', 'binary_crossentropy', metrics=metrics)

In [ ]:
filename = path_chkpt + '/Mortality/TCN_best_model'
checkpoint = ModelCheckpoint(filename, monitor='val_auc',verbose=1, save_best_only=True, mode='max')
print('Train...')
model.fit(
    X_train, y_train,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint],
    epochs=10
)

MRM

In [ ]:
def MRM_model(input_shape, hidden = 256, targets = 1, learn_rate = 1e-4):
    model = Sequential()
    model.add(Bidirectional(LSTM(hidden), merge_mode = 'concat'))
    model.add(Dropout(0.5))
    model.add(Dense(targets))
    model.add(Activation ('sigmoid'))
    model.compile(loss='binary_crossentropy', 
    optimizer=Adam(lr=learn_rate, beta_1 =.5 ), metrics=['accuracy'])
    return (model)

In [ ]:
MRM_model = MRM_model(input_shape=input_shape, targets=1, hidden = 256, learn_rate=1e-4)

In [ ]:
filename = path_chkpt + '/Mortality/mrm_best_model'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [ ]:
MRM_model.fit(x = X, y= y, validation_split=.2, epochs = 10, verbose=2, batch_size=64, callbacks=[checkpoint])

In [ ]:
prediction = MRM_model.predict_proba(X_test,batch_size=64)

In [ ]:
fpr, tpr, _ = skmetrics.roc_curve(y_test,  prediction)
auc = skmetrics.roc_auc_score(y_test, prediction)

In [ ]:
auc, average_precision_score(y_test, prediction)